In [ ]:
!pip install pyspark

In [ ]:
import json
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, array
from pyspark.sql.types import StructType,ArrayType

In [ ]:
from ast import Mult


def flatten_json(df):
  """Flattens a JSON DataFrame into a single column."""

  # Get the list of all nested columns.
  flattened_columns = []


  for column in df.columns:
    if isinstance(df[column].dataType, StructType):

      flattened_columns.extend(column + ".*" for column in df[column].names)
    if isinstance(df[column].dataType, ArrayType):
      for child_field in df[column].dataType.fields:
        if type(child_field.dataType) == "array":
          flattened_columns["*%d.%s" % (i, child_field.name)] = explode(col(df[column].dataType.name + "." + child_field.name))
        else:
          flattened_columns["*%d.%s" % (i, child_field.name)] = col(df[column].dataType.name + "." + child_field.name)
        i += 1

  # Create a new column with all of the nested columns as an array.
  flattened_column = array(*flattened_columns)

  # Add the flattened column to the DataFrame.
  df = df.withColumn("flattened", flattened_column)

  # Return the flattened DataFrame.
  return df

if __name__ == "__main__":
  # Create a Spark session.
  spark = SparkSession.builder.appName("Flatten JSON").getOrCreate()

  # Read the JSON file into a DataFrame.
  df = spark.read.json("/content/exampleEliteJSON.json",multiLine=True)

  # Flatten the DataFrame.
  flattened_df = flatten_json(df)

  # Print the flattened DataFrame.
  flattened_df.show()


+--------------------+------------+--------------------+--------------------+--------------------+-------------------------+------------------------------+----------------+--------------------+--------------+------+--------------------+-----------------------+--------------------+----------------+------------+---------------+------+--------+---------------+--------------+--------------------+---------------+---------------------+--------------------+-------------------+---------------------------+------------------------+----------------+---------+
|            AgencyID|AgencyNumber|        AuditEnabled|     CreatedByAuthor|    FinishedIncident|FormHierarchyCollectionID|IncidentIncidentNumberModValue|IncidentOriginID|        IncidentTime|IncidentTypeID|Locked|                 OID|PatientCareReportNumber|               Scene| SoftwareCreator|SoftwareName|SoftwareVersion|Status|Validity|__lake_clientId|__lake_dataset| __lake_lastModified|__lake_sourceId|__lake_sourcePipeline|__lake_sourc

In [1]:
import json
import pandas as pd

# Load the JSON data into a Python dictionary
json_data = json.load(open('/content/exampleEliteJSON.json'))


# Initialize the nested_columns variable
nested_columns = []

# Flatten the nested JSON object into a DataFrame
df = pd.json_normalize(json_data,max_level=10)

# Identify the columns with nested JSON objects
for column in df.columns:
  if isinstance(df[column][0], list):
        nested_columns.append(column)

print(nested_columns)
# Explode the columns with nested JSON objects
for column in nested_columns:
    df[column] = df[column].apply(pd.json_normalize, max_level=5)

    # Create new columns for the nested values
    for key, value in df[column][0].items():
        df[f'{column}_{key}'] = value

# Print the DataFrame
df.head()


['Scene.Response.Patient.Billing.BillingInsurances', 'Scene.Response.Patient.CardiacArrest.AEDPriorToArrivals', 'Scene.Response.Patient.CardiacArrest.CardiacArrestWitnesses', 'Scene.Response.Patient.CardiacArrest.CardiacRhythmAtDestinations', 'Scene.Response.Patient.CardiacArrest.CirculationReturneds', 'Scene.Response.Patient.CardiacArrest.CPRPerformeds', 'Scene.Response.Patient.CardiacArrest.CPRPriorToArrivals', 'Scene.Response.Patient.CardiacArrest.ResuscitationAttempts', 'Scene.Response.Patient.Disposition.AdditionalTransportModes', 'Scene.Response.Patient.Disposition.DestinationReasons', 'Scene.Response.Patient.PatientSituation.PatientComplaints', 'Scene.Response.Patient.AdvanceDirectives', 'Scene.Response.Patient.CurrentMedications', 'Scene.Response.Patient.EnvironmentAllergies', 'Scene.Response.Patient.Exams', 'Scene.Response.Patient.MedicalHistoryObtaineds', 'Scene.Response.Patient.MedicationAllergies', 'Scene.Response.Patient.Medications', 'Scene.Response.Patient.MedSurgHistori

<ipython-input-1-2e77e0ee30fc>:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{column}_{key}'] = value
<ipython-input-1-2e77e0ee30fc>:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{column}_{key}'] = value
<ipython-input-1-2e77e0ee30fc>:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`


,AgencyID,AuditEnabled,CreatedByAuthor,FormHierarchyCollectionID,IncidentOriginID,IncidentTypeID,Locked,PatientCareReportNumber,SoftwareCreator,SoftwareName,...,Scene.Response.CrewMembers_CrewMemberRoles,Scene.Response.CrewMembers_CrewMemberAgencyPerformerIDModValue.LicensureID,Scene.Response.CrewMembers_CrewMemberAgencyPerformerIDModValue.PerformerID,Scene.Response.CrewMembers_CrewMemberCrewMemberLevelModValue.CrewMemberLevel,Scene.Response.DispatchDelays_DispatchDelayDelayReasonModValue.DelayReason,Scene.Response.ResponseModes_ResponseModeResponseModeModValue.ResponseMode,Scene.Exposures_LicensureID,Scene.Exposures_ExposureProtectiveEquipments,Scene.Exposures_ExposureTypes,Scene.Exposures_ExposureSuspectedExposureModValue.SuspectedExposure
0,4f296634-142c-40d5-9c9a-fc6e81c5baea,2020-04-06T17:28:26.603,b5107cc0-16f8-4a30-82e6-b8a1f0cbf23a,1038,1,1,True,1cdf47a3a2fb4e2dafb276e851d0c6c0,"ImageTrend, Inc.",Elite,...,[{'CrewMemberRoleRoleIDModValue': {'RoleID': '...,1122334455,ed8ad2d5-f0a4-447f-b3ef-850e40ec9f7b,CrewMemberCertificationLevel_EMTParamedic,DispatchDelayType_NoneNoDelay,ResponseModeDescriptor_LightsAndSirens,1122334455,[{'ProtectiveEquipment': 'PersonalProtectiveEq...,[{'ExposureTypeExposureTypeModValue': {'NotVal...,EMSWorkRelatedHarmModifier_No


In [ ]:
df.to_csv("test_pandas.csv")